In [18]:
import ts from "npm:typescript";

console.log("Imported")

Imported


In [27]:
const entryFile = "../src/lib_importer.ts";
const targetFunction = "add2";

In [34]:
const filePath = entryFile;

const fileContent = await Deno.readTextFile(filePath);
const program = ts.createProgram([filePath], {});
const checker = program.getTypeChecker();
const sourceFile = program.getSourceFile(filePath);
// const sourceFile = ts.createSourceFile(filePath, fileContent, ts.ScriptTarget.Latest);
sourceFile.fileName

"../src/lib_importer.ts"

In [49]:
/**
 * Recursively creates a JSON object for functions, methods, and their calls.
 * @param node The current AST node to analyze.
 * @param checker The TypeChecker instance for type analysis.
 * @returns A JSON object representing the function or method, or `null` if not applicable.
 */
function createFunctionAndMethodJson(node: ts.Node, checker: ts.TypeChecker): any {
  let result: any = null;

  // Check if the node is a function or method
  if (
    ts.isFunctionDeclaration(node) ||
    ts.isMethodDeclaration(node) ||
    ts.isFunctionExpression(node) ||
    ts.isArrowFunction(node)
  ) {
    const type = checker.getTypeAtLocation(node);
    result = {
      kind: ts.SyntaxKind[node.kind],
      name: (node.name && node.name.getText()) || "anonymous",
      text: node.getText(),
      type: type ? checker.typeToString(type) : "unknown",
      parameters: node.parameters?.map((param) => ({
        name: param.name.getText(),
        type: param.type ? checker.typeToString(checker.getTypeAtLocation(param)) : "unknown",
      })),
      calls: [],
    };

    // Analyze the function/method body for function/method calls
    if (node.body) {
      ts.forEachChild(node.body, (child) => {
        console.log("child", child);
        if (ts.isCallExpression(child)) {
          const calledFunction = child.expression.getText();
          const callType = checker.getTypeAtLocation(child.expression);
          result.calls.push({
            calledFunction,
            type: callType ? checker.typeToString(callType) : "unknown",
          });
        }
      });
    }
  }

  // Traverse child nodes and record any functions, methods, or calls
  ts.forEachChild(node, (child) => {
    const childJson = createFunctionAndMethodJson(child, checker);
    if (childJson) {
      result = result || { kind: "Parent", children: [] };
      result.children = result.children || [];
      result.children.push(childJson);
    }
  });

  return result;
}

In [50]:
createFunctionAndMethodJson(sourceFile, checker);

child <ref *4> NodeObject {
  pos: 85,
  end: 105,
  kind: 253,
  id: 0,
  flags: 0,
  modifierFlagsCache: 0,
  transformFlags: 4194432,
  parent: <ref *3> NodeObject {
    pos: 83,
    end: 107,
    kind: 241,
    id: 0,
    flags: 0,
    modifierFlagsCache: 0,
    transformFlags: 4194432,
    parent: <ref *1> NodeObject {
      pos: 31,
      end: 107,
      kind: 262,
      id: 13,
      flags: 0,
      modifierFlagsCache: 536870944,
      transformFlags: 4194433,
      parent: <ref *2> SourceFileObject {
        pos: 0,
        end: 108,
        kind: 307,
        id: 0,
        flags: 0,
        modifierFlagsCache: 0,
        transformFlags: 129,
        parent: undefined,
        original: undefined,
        emitNode: undefined,
        statements: [
          [NodeObject],
          [Circular *1],
          pos: 0,
          end: 107,
          hasTrailingComma: false,
          transformFlags: 129
        ],
        endOfFileToken: TokenObject {
          pos: 107,
          en

{
  kind: "Parent",
  children: [
    {
      kind: "FunctionDeclaration",
      name: "add2",
      text: "export function add2(a: number, b: number): number {\n" +
        "  return add(a, b);\n" +
        "}",
      type: "(a: number, b: number) => number",
      parameters: [ { name: "a", type: "number" }, { name: "b", type: "number" } ],
      calls: []
    }
  ]
}